# Imports

In [1]:
import re 
import pandas as pd
import os
from bs4 import BeautifulSoup
import codecs

# Chemins des fichiers

In [2]:
#mettre les chemins des dossiers
path_lemonde_diplomatique = 'F:\Corpus\Le_Monde_2\W0036-02'
path_lemonde = 'F:\Corpus\Le_Monde_2\W0015'

In [3]:
def convert_encoding(input_file, output_file):
    """Pour convertir un fichier iso en utf8"""
    with codecs.open(input_file, 'r', encoding='ISO-8859-15') as file:
        content = file.read()
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(content)

In [4]:
def extract_files(folder_path):
    """Extract all files in folder"""
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_paths.append(os.path.join(root, file))
    return file_paths

# Extraction de tous les fichiers + garder ceux avec les extensions qui nous intéressent

In [5]:
#pour le monde diplomatique
files = extract_files(path_lemonde_diplomatique)
files = [x for x in files if x.endswith('-article.html')]

In [6]:
#pour le monde 
files2 = extract_files(path_lemonde)
files2 = [x for x in files2 if x.endswith('UTF8.xml')]

In [7]:
def read_html_file(file_path):
    try : 
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        soup = BeautifulSoup(html_content, 'html.parser')
    except UnicodeDecodeError:
        print('read_html_file erreur : ', file_path)
        soup = None
    return soup

In [8]:
def read_XML_file(file_path):
    print(file_path)
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
        soup = BeautifulSoup(html_content, 'xml')
    return soup

In [47]:
def get_numb(soup):
    all_notes = soup.find_all('p', attrs={'class':'spip_note'})
#     numbers = {}
#     for spip_note in all_notes: 
#         a_tag = spip_note.find('a', class_='spip_note')
#         if a_tag is not None:
#             number = a_tag.get_text().strip()
#             numbers['{}'.format(number)] = spip_note.get_text().replace('\xa0',' ')
#     return numbers

In [48]:
def replace(replacement_dict, match):
    number = match.group(1)
    if number in replacement_dict:
        return replacement_dict[number]
    return match.group(0)

In [49]:
def get_spip(soup, text):
    new_str = []
    replacement_dict = get_numb(soup)
    pattern = '\(([0-9]*)\)'
    for word in text.split():
        match = re.match(pattern, word)
        if match : 
            number = match.group(1)
            if number in replacement_dict:
                replacement = '['+ replacement_dict[number] +']'
                word = re.sub(pattern, replacement, word)
                new_str.append(word)
            else:
                new_str.append(word)
        else : 
            new_str.append(word)
    #text = re.sub(pattern, replace(replacement_dict, match), text)
    #for match in matches:
     #   print(match)
      #  if match in replacement_dict:
       #     replacement = replacement_dict[match]
        #    print(replacement)
         #   print('---')
         #   text = text.replace(f'({match})', replacement)
    return new_str

In [152]:
def extract_data(soup, file):
    title = soup.find('title').get_text()
    authors_element = soup.find('meta', attrs={'name': 'Authors'})
    try :
        keywords = soup.find('meta', attrs={'name': 'Keywords'}).get('content')
    except AttributeError:
        keywords = ''
    authors = authors_element['content'] if authors_element else None
    text = soup.find('span', attrs={'class':'corpsText'}).get_text().replace('\xa0',' ')#.decode('utf-8')
    all_notes = ' '.join([x.get_text().strip() for x in soup.find_all('p', attrs={'class':'spip_note'})])
    text = text + all_notes
#     text = ' '.join(get_spip(soup, text))
    pat = r'F:\\Corpus\\Le_Monde_2\\W0036-02\\([^\\]*)\\([^\\]*)\\([^\\]*)-article\.html'
    match = re.search(pat, file)
    name = match.group(1)+'_'+match.group(2)+'_'+match.group(3)
    try :
        date = soup.find('td', attrs={'class': 'date'}).get_text().replace('\n','').replace('  ','')
    except:
        date = match.group(2)+'/'+ match.group(1)
    dict_ = {}
    dict_['name']=name
    dict_['author'] = authors
    dict_['mots_clefs'] = keywords
    dict_['date'] = date
    dict_['text'] = clean(fix_encoding(text)).strip()
    return dict_

In [153]:
def extract_data_XML(soup, file):
    title = fix_encoding(soup.find('Titre').get_text())
    authors = fix_encoding(soup.find('SignaturesIndexees').get_text()).strip()
    jour = soup.find('Date').get('Jour')
    mois = soup.find('Date').get('Mois')
    annee = soup.find('Date').get('Annee')
    date = str(jour +'/'+ mois +'/'+ annee)
    text = soup.find('Texte').get_text().strip()
    text = fix_encoding(text)
    
    if authors == '':
        pat2 = ".*\. - \((.*)\)$"
        match2 = re.search(pat2, text)
        if match2:
            authors = match2.group(1).replace('.','')
            
    pat = r'F:\\Corpus\\Le_Monde_2\\W0015\\2007\\([^\\]*)\\([^\\]*)\\([^\\]*)\.xml'
    match = re.search(pat, file)
    if match :
        name = match.group(1)+'_'+match.group(2)+'_'+match.group(3)
    else: 
        pat1 = r'F:\\Corpus\\Le_Monde_2\\W0015\\2007\\([^\\]*)\\([^\\]*)\.xml'
        match1 = re.search(pat1, file)
        name = '0_'+match1.group(1)+'_'+match1.group(2)
   
    categories = ", ".join([x.get_text() for x in soup.find_all('Categorie')])
    
    dict_ = {}
    dict_['name']=name
    dict_['titre']= title
    dict_['categories'] = categories
    dict_['author'] = authors
    dict_['date'] = date
    dict_['text'] = clean(fix_encoding(text)).strip()
    return dict_

In [154]:
def create_dict(file_path, ext):
    if ext =="XML":
        soup = read_XML_file(file_path)
        if soup:
            #dict_ =extract_data(soup, file_path)
            dict_ =extract_data_XML(soup, file_path)
            return dict_
        else:
            return None
    elif ext =="HTML":
        soup = read_html_file(file_path)
        if soup:
            dict_ =extract_data(soup, file_path)
            return dict_
        else:
            return None

In [155]:
def clean(text):
    text = re.sub('\(\d*\)', '', text)
    text = re.sub('^\"', '', text)
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
    text = re.sub('«', '', text)
    text = re.sub('»', '', text)
    text = re.sub('\s-\s\([^\.]*\.\)$', '', text)
    text = re.sub('\"', '', text)
    text = text.replace('"', '')
    text = text.replace('\"', '')
    text = text.replace('“', '')
    text = text.replace('”', '')
    text = text.replace('\n', '')
    text = text.replace('*', '')
    text = text.replace('_','')
    text = text.replace(' – ', ' ,')
    text = text.replace('\xad','')
    return text 

In [156]:
'ursuite (2 Ã 10 km), p'

'ursuite (2 Ã\x97 10 km), p'

In [157]:
def fix_encoding(input_string):
    encoding_dict = {
        'Ã©': 'é',
        'Ãš': 'è',
        'Â«': '«',
        'Ã\xa0':'à',
        'Ã\x97':'à',
        'Â\x92':"\'",
        'Â»':'»',
        'Ã®':'î',
        'Ã\x9b':'Û',
        'Ãª':'ê',
        'Ã\x82':'Â',
        "Ã\xad": "í",
        "Â\x96":',',
        'Ã¯':'ï',
        'Ã¢':'â',
        'Ã§':'ç',
        'Ã¹':'ù',
        "Ã½": "ý",
        "Ã¼": "ü",
        'Ã»':'û',
        '\x9c':'œ',
        '\x8c':'œ',
        '\x93': '',
        '\x94':'',
        '\x1a':'',
        'ÃŽ':'ô',
        'Â\x80':' ',
        'Ã«':'ë',
        'Ã\x89':'É',
        'Ã\x80':'À',
        "ÃŸ": "ß",
        'Ã\x94':'Ô',
        'Â\x8c':'Œ',
        'Ã¶':'ö',
        'Ã\x87':'Ç',
        r'\\\'': '\'',
        'Ã\x88':'È',
        'Ã\x8a':'Ê',
        'ÃŒ':'ü',
        'Ã\x8e':'Î',
        'Â°':'°',
        "Ã\x8f":"Ï",
        "Ã±": "ñ",
        "Ã³": "ó",
        "Ã´": "ô",
        "Ã¶": "ö",
        "\xa0":"",
    }

    for encoded_char, correct_char in encoding_dict.items():
        input_string = input_string.replace(encoded_char, correct_char)

    return input_string

In [158]:
def index2article(index):
    soup = read_html_file(index)
    articles_link = [x.get('href') for x in soup.find_all('a', attrs={'class': 'devtextesommaire'})]
    news = [index.replace('index.html','{}'.format(x)).replace('/','\\') for x in articles_link]
    resume = soup.find('div', attrs={'class':'diplozizi'})
    list_dict = []
    i = 0
    for x in resume.find_all('p', attrs={'class':'spip'}):
        dico = {}
        i+=1
        if i%2!=0:
            dico['name']=x.get_text()
        elif i%2==0:
            dico['text']=x.get_text().replace('&nbsp', ' ')
            dico['categorie']='Resumé 1ère page'
            dico['date'] = re.sub('F:/Corpus/Le_Monde_2/W0036-02/(\d*)/(\d*)/index.html','\2-\1',index)
            list_dict.append(dico)
        
    dict1={}
    for x in news :
        dict_ = {}
        soup1 = read_html_file(x)
        title = soup1.find('title').get_text()
        authors_element = soup1.find('meta', attrs={'name': 'Authors'})
        authors = authors_element['content'] if authors_element else None
        text = soup1.find('p', attrs={'class':'spip'}).get_text().replace('\xa0',' ')#.decode('utf-8')
        all_notes = ' '.join([x.get_text().strip() for x in soup1.find_all('p', attrs={'class':'spip_note'})])
        text = text + all_notes
#         text = ' '.join(get_spip(soup1, text))
        pat = r'F:\\Corpus\\Le_Monde_2\\W0036-02\\(2004)\\([^\\]*)\\([^\\]*)\\([^\\]*)'
        match = re.search(pat, x)
        name = match.group(1)+'_'+match.group(2)+'_'+match.group(3)+'_'+match.group(4)
        date = soup1.find('meta', attrs={'name': 'Date'}).get('content')
        rubrique = soup1.find('meta', attrs={'name': 'Rubrique'}).get('content')
        try : 
            themes = soup1.find('meta', attrs={'name': 'Keywords'}).get('content')
        except AttributeError:
            themes = '' 
        dict_['name']= name
        dict_['categorie']= rubrique
        dict_['mots_clefs']=themes
        dict_['author'] = authors
        dict_['date'] = date
        dict_['text'] = clean(fix_encoding(text))
        list_dict.append(dict_)
    return list_dict

# Analyse + création d'un df et d'un fichier csv

In [128]:
#pour le monde diplomatique (sauf 2004)
list_dict = [create_dict(x,'XML') for x in files2]
new = [x for x in list_dict if x]
df = pd.DataFrame(new)

F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0001UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0002UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0003UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0004UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0005UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0006UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0007UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0008UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0009UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0010UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0011UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0012UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0013UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0014UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0015UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0016UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20

F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0077UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0078UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0079UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0080UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0081UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0082UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0083UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0084UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0085UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0086UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0087UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0088UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0089UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0090UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0092UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0093UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20

F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0035UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0036UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0037UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0038UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0039UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0040UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0041UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0042UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0043UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0044UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0045UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0046UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0047UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0048UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0049UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0050UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20

F:\Corpus\Le_Monde_2\W0015\2007\01\05\LIV20070105N0016UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\LIV20070105N0017UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\LIV20070105N0018UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\LIV20070105N0019UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\LIV20070105N0020UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\LIV20070105N0021UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\LIV20070105N0022UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\LIV20070105N0023UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\LIV20070105N0024UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\LIV20070105N0025UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\LIV20070105N0026UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\LIV20070105N0027UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\LIV20070105N0028UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\LIV20070105N0029UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\LIV20070105N0030UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\LIV20070105N0031U

F:\Corpus\Le_Monde_2\W0015\2007\01\08\20070107N0007UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\20070107N0008UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\20070107N0009UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\20070107N0010UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\20070107N0011UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\20070107N0012UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\20070107N0013UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\20070107N0014UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\20070107N0015UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\20070107N0016UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\20070107N0018UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\20070107N0019UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\20070107N0020UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\20070107N0021UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\20070107N0022UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\20070107N0023UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\20

...

F:\Corpus\Le_Monde_2\W0015\2007\12\MDE20071113N0024UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\MDE20071113N0025UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\MDE20071113N0026UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\MDE20071113N0027UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\MDE20071113N0028UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\MDE20071113N0029UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\MDE20071113N0030UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\MDE20071113N0031UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\MDE20071113N0032UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\MDE20071113N0033UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\MDE20071113N0034UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\MDE20071113N0035UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\MDE20071113N0036UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\MDE20071120N0001UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\MDE20071120N0002UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\MDE20071120N0003UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\MDE20

F:\Corpus\Le_Monde_2\W0015\2007\12\SPA20071010N0010UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\SPA20071010N0011UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\SPA20071010N0012UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\SPA20071010N0013UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\SPA20071010N0014UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\SPA20071010N0015UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\SPA20071010N0016UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\SPA20071010N0017UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\SPA20071108N0001UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\SPA20071108N0002UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\SPA20071108N0003UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\SPA20071108N0004UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\SPA20071108N0005UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\SPA20071108N0006UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\SPA20071108N0007UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\SPA20071108N0008UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\SPA20

F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20070930N0020UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20070930N0021UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20070930N0022UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20070930N0023UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20070930N0024UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20070930N0025UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20070930N0026UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20070930N0027UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20070930N0028UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071007N0001UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071007N0002UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071007N0003UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071007N0004UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071007N0005UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071007N0006UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071007N0007UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20

F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071111N0024UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071111N0025UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071111N0026UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071111N0027UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071111N0028UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071111N0029UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071111N0030UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071111N0031UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071111N0032UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071118N0001UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071118N0002UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071118N0003UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071118N0004UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071118N0005UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071118N0006UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071118N0007UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20

F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071230N0005UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071230N0006UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071230N0007UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071230N0008UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071230N0009UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071230N0010UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071230N0011UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071230N0012UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071230N0013UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071230N0014UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071230N0015UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071230N0016UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071230N0017UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071230N0018UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071230N0019UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20071230N0020UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\12\TEL20

In [129]:
df

,name,titre,categories,author,date,text
0,01_01_20061231N0001UTF8,Saddam Hussein a été pendu à Bagdad pour crime...,"EVENEMENT, MANCHETTE, IMPORTANT, PHOTO, OUVERT...",Cécile Hennion (avec AFP\n AP\n Reuters),01/01/2007,"Trois ans ou presque, jour pour jour, après sa..."
1,01_01_20061231N0002UTF8,Jacques Chirac entend peser sur la campagne,"ACCROCHE, DESSIN",Plantu,01/01/2007,"Dimanche 31 décembre, à 20 heures, le présiden..."
2,01_01_20061231N0003UTF8,Les filières qui assureront un emploi dans dix...,ACCROCHE,,01/01/2007,Quelle orientation choisir pour avoir un métie...
3,01_01_20061231N0004UTF8,"Face au chaos, le président somalien réclame d...","ACCROCHE, PHOTO",,01/01/2007,"Il faut soutenir le peuple, qui souffre depui..."
4,01_01_20061231N0005UTF8,Non à la peine de mort,"EDITORIAL, IMPORTANT",,01/01/2007,"Pas grand-monde, sauf peut-être dans son fief ..."
...,...,...,...,...,...,...
44285,0_12_TEL20071230N0029UTF8,"Kitty Harris, une vie d'espionne.","CRITIQUE, PHOTO, PORTRAIT",Francis Cornu,31/12/2007,Née à Londres dans une famille ouvrière juive ...
44286,0_12_TEL20071230N0030UTF8,Les mystères de L'Arche de Zoé,"CRITIQUE, PHOTO",Olivier Zilbertin,31/12/2007,Le document que propose ce soir le magazine P...
44287,0_12_TEL20071230N0031UTF8,Dans le monde de Jacques Prévert,"CRITIQUE, PHOTO, PORTRAIT",Gérard Condé,31/12/2007,"Seuls les critiques, disait-il, savent ce que..."
44288,0_12_TEL20071230N0032UTF8,La Belle Espionne,CRITIQUE,Jacques Siclier,31/12/2007,"En 1803, Napoléon (Gérard Oury) s'apprête à en..."


df.to_csv(r'F:\Corpus\finaux\lemonde2003_essai.tsv', sep='\t', encoding='utf-8', index=False)

In [108]:
import codecs
try:
    f = codecs.open(r"F:\Corpus\finaux\Karo\lemonde2007_3col.tsv", encoding='utf-8', errors='strict')
    for line in f:
        pass
    print("Valid utf-8")
except UnicodeDecodeError:
    print("invalid utf-8")

Valid utf-8


In [130]:
df.to_csv('F:\Corpus\\finaux\\lemonde2007.tsv', sep='\t', encoding='utf-8', index=False)

In [131]:
lemonde2007= df[['name','author','text']]
lemonde2007 = lemonde2007.rename(columns={'name': 'nom', 'author': 'auteur', 'text':'text_clean'})
lemonde2007.to_csv(r'F:\Corpus\finaux\Karo\lemonde2007_3col.tsv', sep='\t', encoding='utf-8', index=False)

In [162]:
#pour le monde diplomatique 2004
path2004 = r'F:\Corpus\Le_Monde_2\W0036-02\2004'
files2004 = extract_files(path2004)
files2004 = [x for x in files2004 if x.endswith('.html')]
listlist = [index2article(x) for x in files2004]

In [163]:
flatten_data = [item for sublist in listlist for item in sublist]
# Convert to dataframe
df2004 = pd.DataFrame(flatten_data, columns=['name', 'categorie', 'author', 'date', 'text'])

In [164]:
df2004 = df2004.iloc[3:]

In [165]:
df2004

,name,categorie,author,date,text
3,2004_01_RAMONET_10615,Archives,Ignacio Ramonet,2004-01-01,"A Genève, durant trois jours, du 10 au 12 déce..."
4,2004_01_CONCHIGLIA_10996,Archives,Augusta Conchiglia,2004-01-01,"Prévisible, l’arrestation de M. Saddam Hussein..."
5,2004_01_A_10932,Archives,None,2004-01-01,L’assemblée constitutive de l’Observatoire fra...
6,2004_01_WINCKLER_10693,Archives,Martin Winckler,2004-01-01,Le Haut Conseil pour l’avenir de l’assurance-m...
7,2004_01_MASCLET_10698,Archives,Olivier Masclet,2004-01-01,Annonçant la future loi contre le port de sign...
...,...,...,...,...,...
358,2004_12_JELINEK_11741,Archives,Elfriede Jelinek,2004-12-01,Faut-il se mettre en colère face au besoin de ...
359,2004_12_CHANDA_11746,Archives,Tirthankar Chanda,2004-12-01,Longtemps marquée par la négritude et l’engag...
360,2004_12_A_11747,Archives,None,2004-12-01,"Kangni Alem, Coca Cola Jazz, Dapper, Paris, 20..."
361,2004_12_AMOURO_11748,Archives,Camille Amouro,2004-12-01,Pierre Casanova Le Même pète tous les soirs à ...


In [166]:
df2004.to_csv(r'F:\Corpus\finaux\lemonde2004.csv', sep='\t', encoding='utf-8', index=False)

In [167]:
df2004_2 = df2004[['name','author','text']]

df2004_2 = df2004_2.rename(columns={'name': 'nom', 'author': 'auteur', 'text':'text_clean'})
df2004_2.to_csv(r'F:\Corpus\finaux\Karo\lemonde2004_3col.tsv', sep='\t', encoding='utf-8', index=False)

In [170]:
files2

['F:\\Corpus\\Le_Monde_2\\W0015\\2007\\01\\01\\20061231N0001UTF8.xml',
 'F:\\Corpus\\Le_Monde_2\\W0015\\2007\\01\\01\\20061231N0002UTF8.xml',
 'F:\\Corpus\\Le_Monde_2\\W0015\\2007\\01\\01\\20061231N0003UTF8.xml',
 'F:\\Corpus\\Le_Monde_2\\W0015\\2007\\01\\01\\20061231N0004UTF8.xml',
 'F:\\Corpus\\Le_Monde_2\\W0015\\2007\\01\\01\\20061231N0005UTF8.xml',
 'F:\\Corpus\\Le_Monde_2\\W0015\\2007\\01\\01\\20061231N0006UTF8.xml',
 'F:\\Corpus\\Le_Monde_2\\W0015\\2007\\01\\01\\20061231N0007UTF8.xml',
 'F:\\Corpus\\Le_Monde_2\\W0015\\2007\\01\\01\\20061231N0008UTF8.xml',
 'F:\\Corpus\\Le_Monde_2\\W0015\\2007\\01\\01\\20061231N0009UTF8.xml',
 'F:\\Corpus\\Le_Monde_2\\W0015\\2007\\01\\01\\20061231N0010UTF8.xml',
 'F:\\Corpus\\Le_Monde_2\\W0015\\2007\\01\\01\\20061231N0011UTF8.xml',
 'F:\\Corpus\\Le_Monde_2\\W0015\\2007\\01\\01\\20061231N0012UTF8.xml',
 'F:\\Corpus\\Le_Monde_2\\W0015\\2007\\01\\01\\20061231N0013UTF8.xml',
 'F:\\Corpus\\Le_Monde_2\\W0015\\2007\\01\\01\\20061231N0014UTF8.xml',
 'F:\\

In [169]:
#pour le monde 2007
list_dict2 = [create_dict(x,'XML') for x in files2]
df2 = pd.DataFrame(list_dict2)
df2.to_csv(r'F:\Corpus\finaux\lemonde2007.tsv', sep='\t', encoding='utf-8', index=False)

F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0001UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0002UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0003UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0004UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0005UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0006UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0007UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0008UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0009UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0010UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0011UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0012UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0013UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0014UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0015UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20061231N0016UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\01\20

F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0084UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0085UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0086UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0087UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0088UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0089UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0090UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0092UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0093UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0094UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0095UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0096UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0097UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0098UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0099UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20070102N0100UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\02\20

F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0081UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0082UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0083UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0084UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0085UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0086UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0087UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0088UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0089UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0090UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0091UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0092UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0093UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\04\20070104N0094UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\20070105N0001UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\20070105N0002UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\05\20

F:\Corpus\Le_Monde_2\W0015\2007\01\06\20070106N0007UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\06\20070106N0008UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\06\20070106N0009UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\06\20070106N0010UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\06\20070106N0011UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\06\20070106N0012UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\06\20070106N0013UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\06\20070106N0014UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\06\20070106N0015UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\06\20070106N0016UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\06\20070106N0017UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\06\20070106N0018UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\06\20070106N0019UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\06\20070106N0020UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\06\20070106N0021UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\06\20070106N0022UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\06\20

F:\Corpus\Le_Monde_2\W0015\2007\01\08\TEL20070107N0003UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\TEL20070107N0004UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\TEL20070107N0005UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\TEL20070107N0006UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\TEL20070107N0007UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\TEL20070107N0008UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\TEL20070107N0009UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\TEL20070107N0010UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\TEL20070107N0011UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\TEL20070107N0012UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\TEL20070107N0013UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\TEL20070107N0014UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\TEL20070107N0015UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\TEL20070107N0016UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\TEL20070107N0017UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\01\08\TEL20070107N0018U

...

F:\Corpus\Le_Monde_2\W0015\2007\02\16\20070216N0012UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\16\20070216N0013UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\16\20070216N0014UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\16\20070216N0015UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\16\20070216N0016UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\16\20070216N0017UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\16\20070216N0018UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\16\20070216N0019UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\16\20070216N0020UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\16\20070216N0021UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\16\20070216N0022UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\16\20070216N0023UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\16\20070216N0024UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\16\20070216N0025UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\16\20070216N0026UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\16\20070216N0027UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\16\20

F:\Corpus\Le_Monde_2\W0015\2007\02\17\20070217N0012UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\17\20070217N0013UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\17\20070217N0014UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\17\20070217N0015UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\17\20070217N0016UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\17\20070217N0017UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\17\20070217N0018UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\17\20070217N0019UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\17\20070217N0020UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\17\20070217N0021UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\17\20070217N0022UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\17\20070217N0023UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\17\20070217N0024UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\17\20070217N0025UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\17\20070217N0026UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\17\20070217N0027UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\17\20

F:\Corpus\Le_Monde_2\W0015\2007\02\19\20070218N0089UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\19\20070218N0090UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\19\20070218N0091UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\19\20070218N0093UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\19\20070218N0094UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\19\20070218N0095UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\19\20070218N0096UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\19\20070218N0097UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\19\20070218N0098UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\19\20070218N0099UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\19\20070218N0100UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\19\20070218N0101UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\19\20070218N0102UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\19\20070218N0103UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\19\20070218N0104UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\19\20070218N0105UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\19\20

F:\Corpus\Le_Monde_2\W0015\2007\02\20\20070220N0111UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\20\20070220N0112UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\20\20070220N0113UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\20\20070220N0114UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\20\20070220N0115UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\20\20070220N0116UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\20\20070220N0117UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\20\20070220N0118UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\20\20070220N0119UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\20\20070220N0121UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\20\MDE20070220N0001UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\20\MDE20070220N0002UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\20\MDE20070220N0003UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\20\MDE20070220N0004UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\20\MDE20070220N0005UTF8.xml
F:\Corpus\Le_Monde_2\W0015\2007\02\20\MDE20070220N0006UTF8.xml
F:\Corpus\Le_Monde_2\W

KeyboardInterrupt: 

In [ ]:
df2

In [171]:
#pour le monde 2012
lemonde2012 = 'F:\\Corpus\\Le_Monde_2\\W0015\\2012\\LeMonde_20120101-20121231.txt'

In [172]:
def read_file_line_by_line(file_path):
    lines = []
    dict1 = {}
    i = 0
    with open(file_path, 'r') as file:
        ensemble = []
        for line in file:
            if line.strip() == '© SA Le Monde - CEDROM-SNi inc. 2013. Tous droits réservés.':
                if ensemble:
                    dict1[str(i)] = ensemble
                    i += 1
                    ensemble = []
            else:
                ensemble.append(line.strip())
    
    return dict1

In [173]:
dict2012 = read_file_line_by_line(lemonde2012)

In [174]:
def get_sub_lists(data):
    sub_lists = []
    sub_list = []
    count = 0

    for item in data:
        if item != '':
            sub_list.append(item)
            count = 0
        else:
            count += 1
            if count == 3:
                sub_lists.append(sub_list)
                sub_list = []
                count = 0

    # Ajouter la dernière sous-liste
    sub_lists.append(sub_list)
    return sub_lists

def supprimer_debut_texte(chaine):
    mots = chaine.split()
    for i, mot in enumerate(mots):
        if mot.lower() == 'correspondant' or mot.lower() == 'correspondance':
            return ' '.join(mots[i+1:])
    return chaine

In [175]:
def supprimer_debut_texte(chaine):
    mots = chaine.split()
    debut_indices = []
    mots_cles = ['correspondant', 'correspondance']
    mots_cles_composes = ['Envoyé spécial']
    
    for i, mot in enumerate(mots):
        if mot.lower() in mots_cles:
            debut_indices.append(i)
        elif i < len(mots) - 1:
            mot_compose = mot.lower() + ' ' + mots[i + 1].lower()
            if mot_compose in mots_cles_composes:
                debut_indices.append(i)

    if debut_indices:
        debut_index = min(debut_indices)
        return ' '.join(mots[debut_index + 1:])
    return chaine

In [194]:
def extract(data):
    dict_ = {}
    sub_lists = get_sub_lists(data)
    if check_value_occurrences(sub_lists, sub_lists[3], 'N'):
        auteur = sub_lists[3]
        next_ = int(check_value_occurrences(sub_lists, sub_lists[3], 'Y'))
        text = sub_lists[4:next_]
        text = supprimer_debut_texte(' '.join([item for sublist in text for item in sublist]))
    else:
        auteur = ''
        text = ''.join(''.join(sublist) for sublist in sub_lists[3:-3])
    pat = r'([0-9]* [a-z-9áàâäãåçéèêëíìîïñóòôöõúùûüýÿæœ]* [0-9]{4}), .*'
    pat2 = r'(.*):(.*)'
    match = re.search(pat,  sub_lists[1][1])
    if len(sub_lists[2])>1:
        categorie = ''.join(sub_lists[2][0])
        titre = ' '.join(sub_lists[2][1:])
    else:
        categorie = ''
        titre = ' '.join(sub_lists[2])
    dict_['journal']= ''.join(sub_lists[1][0])
    dict_['date']= match.group(1)
    dict_['titre']= titre
    dict_['categorie'] = categorie
    dict_['auteur'] = ' '.join(auteur)
    dict_['texte'] = clean(fix_encoding(text)).replace('\n', '').strip()
    
    for sous_liste in sub_lists[-2]:
        match2 = re.search(pat2, sous_liste)
        dict_['{}'.format(match2.group(1).strip())] = match2.group(2).strip()
    return dict_

In [195]:
def check_value_occurrences(lst_of_lists, value, response):
    count = 0
    index = None
    for i, sublist in enumerate(lst_of_lists):
        if sublist == value:
            count += 1
            if count == 2:
                index = i
                break
    if response == 'Y':
        return index
    else:
        return count == 2

In [196]:
list_dict2012 = [extract(v) for k, v in dict2012.items()]

In [197]:
df2012 = pd.DataFrame(list_dict2012)

In [198]:
df2012 = df2012[['journal', 'date', 'titre', 'categorie', 'auteur', 'texte', 'Section','Taille', 'Type d\'article']]

In [199]:
df2012

,journal,date,titre,categorie,auteur,texte,Section,Taille,Type d'article
0,Le Monde,31 décembre 2012,Afghanistan : l'impossible cocorico,ÉDITORIAL,,Faute de jouer dans la cour des grands sur le ...,A la Une,COURT,NaN
1,Le Monde,31 décembre 2012,À NOS LECTEURS & DIFFUSEURS,,,L'année 2012 aura été émaillée pour tous les t...,A la Une,COURT,NaN
2,Le Monde,31 décembre 2012,13 TOURNANTS MONDIAUX QUI VONT MARQUER 2013,,,"Croissance en Afrique, basculement en Asie, ré...",A la Une,BREF,NaN
3,Le Monde,31 décembre 2012,TREIZE BASCULEMENTS DU MONDE POUR 2013 Big-ban...,L'ÉVÉNEMENT,Éric Béziat,"A première vue, l'information est insignifiant...",Événement,COURT,INFOGRAPHIE
4,Le Monde,31 décembre 2012,TREIZE BASCULEMENTS DU MONDE POUR 2013 1. GEOS...,L'ÉVÉNEMENT,,"Le 14 février 1945, sur le Quincy, un croiseur...",Événement,COURT,NaN
...,...,...,...,...,...,...,...,...,...
40204,Le Monde,3 janvier 2012,Techno couleur rétro,& VOUS,,François BostnavaronUn cadeau de dernière minu...,Rendez-vous,COURT,NaN
40205,Le Monde,3 janvier 2012,"Courriels Polémique - Arméniens, Chypriotes, m...",Ecrans,,Jacques GuillemainVERSAILLESSi la reconnaissan...,Télévision & Radio,BREF,NaN
40206,Le Monde,3 janvier 2012,C'est tout Net! Survivre à 2012,Ecrans,Marlène Duretz,"La fin du monde en 2012? Légende ou pas, pourq...",Télévision & Radio,COURT,NaN
40207,Le Monde,3 janvier 2012,A ne pas manquer sur le Monde.fr,"Pour 2012, des voeux très politiques Elections",,"Le président de la République, Nicolas Sarkozy...",Dernière heure,BREF,NaN


In [200]:
df2012.to_csv(r'F:\Corpus\finaux\lemonde2012.tsv', sep='\t', encoding='utf-8', index=False)

In [201]:
df2012_2 = df2012[['titre','auteur','texte']]

df2012_2 = df2012_2.rename(columns={'titre': 'nom', 'texte':'text_clean'})
df2012_2.to_csv(r'F:\Corpus\finaux\Karo\lemonde2012_3col.tsv', sep='\t', encoding='utf-8', index=False)